# PINN

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [17]:
class PINN(nn.Module):
    def __init__(self, x, t, u0, nu, layers):
        super(PINN, self).__init__()
        self.x = torch.tensor(x).to(device)
        self.t = torch.tensor(t).to(device)
        self.u0 = torch.tensor(u0).to(device)
        self.nu = nu
        self.layers = layers

        self.fc = self.initialize_nn(layers)

    def initialize_nn(self, layers):
        modules = []
        for i in range(1, len(layers) - 1):
            modules.append(nn.Linear(layers[i-1], layers[i]))
            modules.append(nn.Tanh())
        modules.append(nn.Linear(layers[-2], layers[-1]))
        return nn.Sequential(*modules)
    
    def forward(self, x, t):
        X = torch.cat((x, t), dim=0)
        u = self.fc(X)
        return u
    
    def pde_loss(self, x, t):
        u = self.forward(x, t)
        u_x = torch.autograd.grad(u, x, grad_outputs=torch.ones_like(u), create_graph=True)[0]
        u_xx = torch.autograd.grad(u_x, x, grad_outputs=torch.ones_like(u_x), create_graph=True)[0]
        u_t = torch.autograd.grad(u, t, grad_outputs=torch.ones_like(u), create_graph=True)[0]

        f = u_t + u * u_x - self.nu * u_xx
        return f
    
    def initial_condition_loss(self):
        u0_pred = self.forward(self.x, torch.zeros_like(self.x))
        u0_true = torch.tensor(self.u0(self.x))
        return torch.mean((u0_pred - u0_true)**2)
    
    def train(self, nIter):
        optimizer = optim.Adam(self.parameters(), lr=0.001)
        for epoch in range(nIter):
            optimizer.zero_grad()
            pde_loss = self.pde_loss(self.x, self.t)
            ic_loss = self.initial_condition_loss()
            loss = pde_loss + ic_loss
            loss.backward()
            optimizer.step()

            if epoch % 100 == 0:
                print(f'Epoch {epoch + 1}')

    def predict(self, x, t):
        x = torch.tensor(x).to(device)
        t = torch.tensor(t).to(device)
        return self.forward(x, t).detach().numpy()

In [8]:
x = np.linspace(0, 2 * np.pi, 256)
t = np.linspace(0, 1.0, 100)

def u0(x):
    return -np.sin(x)
u_0 = u0(x)

layers = [2, 50, 50, 1]
nu = 0.01

In [18]:
model = PINN(x, t, u_0, nu, layers)

nIter = 1000
model.train(nIter)

TypeError: __init__() missing 1 required positional argument: 'out_features'